In [11]:
from json import load
from os import listdir

test_list = []

for name in listdir("tests"):
    if name.startswith("speedtest") and name.endswith(".json"):
        test_list.append(load(open("tests/" + name)))

In [12]:
import numpy as np

def reject_outliers(data, m=1.75):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d / (mdev if mdev else 1.)
    return data[s < m]

for test in test_list:
    times = [x["time"] for x in test]
    filtered_times = reject_outliers(np.array(times))
    for i in reversed(range(len(test))):
        run = test[i]
        if not run["time"] in filtered_times:
            test.pop(i)

In [13]:
# finger avgs

fingers = [
    ["q", "a", "z"],
    ["w", "s", "x"],
    ["e", "d", "c"],
    ["r", "f", "v", "t", "g", "b"],
    ["y", "h", "n", "u", "j", "m"],
    ["i", "k", ","],
    ["o", "l", "."],
    ["p", ";", "/"],
]

finger_sums = [0 for i in range(8)]
finger_counts = [0 for i in range(8)]

for test in test_list:
    for movement in test:
        for i, finger in enumerate(fingers):
            if movement["start"] in finger:
                finger_sums[i] += movement["time"]
                finger_counts[i] += 1
                break

print("Finger averages:", [sum / count for sum, count in zip(finger_sums, finger_counts)])

Finger averages: [0.13480391839394065, 0.12135909219403088, 0.12151773833802755, 0.1256274805882343, 0.12436576564285635, 0.117880126537036, 0.11550019495744653, 0.13465857063829748]


In [14]:
# row avgs

rows = [
    ["q", "w", "e", "r", "t", "y", "u", "i", "o", "p"],
    ["a", "s", "d", "f", "g", "h", "j", "k", "l", ";"],
    ["z", "x", "c", "v", "b", "n", "m", ",", ".", "/"],
]

def get_row(key):
    for i, row in enumerate(rows):
        if key in row:
            return i

row_sums = [[0 for i in range(3)] for i in range(3)]
row_counts = [[0 for i in range(3)] for i in range(3)]

for test in test_list:
    for movement in test:
        start_row = get_row(movement["start"])
        end_row = get_row(movement["end"])
        if start_row is None:
            continue
        row_sums[start_row][end_row] += movement["time"]
        row_counts[start_row][end_row] += 1

print("Row averages:", [[(sum / count if not count == 0 else 0) for sum, count in zip(row_sum, row_count)] for row_sum, row_count in zip(row_sums, row_counts)])

Row averages: [[0.0836474087500001, 0.1072665234571419, 0.1385214946666658], [0.12882193598507483, 0.10984191579999916, 0.13125030591250014], [0.13937050650617294, 0.09969643743750009, 0.1247347050000025]]


In [16]:
from json import dump

output = {
    "finger": [sum / count for sum, count in zip(finger_sums, finger_counts)],
    "row": [[(sum / count if not count == 0 else 0) for sum, count in zip(row_sum, row_count)] for row_sum, row_count in zip(row_sums, row_counts)]
}

dump(output, open("output.json", "w"), indent=2)